In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from dateutil.relativedelta import relativedelta
from skforecast.ForecasterAutoreg import ForecasterAutoreg
plt.style.use('seaborn-v0_8-darkgrid')

from predict import Predict

In [2]:
df = pd.read_csv('database/DIM_PLAYER_GAMES.csv')
test = list(df['PLAYER'].unique()[:100])
data = df[df['PLAYER'].isin(test)]


In [3]:
w = Predict(df=df, player = 'Sam LaPorta', steps=17, regressor=RandomForestRegressor, lags=17)
w.predict()


c:\Users\012356.CORPORATE\OneDrive - TRIMEDX Holdings, LLC\Desktop\FF_DB\predict.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data["FORECAST_DATE"] = pd.date_range(
c:\Users\012356.CORPORATE\OneDrive - TRIMEDX Holdings, LLC\Desktop\FF_DB\predict.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data["FORECAST_DATE"] = pd.to_datetime(


,PLAYER,ACTUAL SEASON TOTAL,PREDICTED,SEASON MAPE,GAME MAPE,GAME RSME,GAMES PREDICTED,LAGS,REGRESSOR
0,Sam LaPorta,57.5,48.36,0.158957,0.61068,11.338864,4,4,"(DecisionTreeRegressor(max_features=1.0, rando..."


In [4]:
problems = []
for name in test:
    try:
        Predict(
            df=df, player=name, steps=17, regressor=RandomForestRegressor, lags=17
        ).predict()
    except:
        problems.append(name)


c:\Users\012356.CORPORATE\OneDrive - TRIMEDX Holdings, LLC\Desktop\FF_DB\predict.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data["FORECAST_DATE"] = pd.date_range(
c:\Users\012356.CORPORATE\OneDrive - TRIMEDX Holdings, LLC\Desktop\FF_DB\predict.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data["FORECAST_DATE"] = pd.to_datetime(
c:\Users\012356.CORPORATE\OneDrive - TRIMEDX Holdings, LLC\Desktop\FF_DB\predict.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

In [5]:
problems

[]

In [9]:
top_100_preds = [
    Predict(
        df=df, player=name, steps=17, regressor=RandomForestRegressor, lags=17
    ).predict()
    for name in test
]

df_top_100 = pd.concat(top_100_preds)

c:\Users\012356.CORPORATE\OneDrive - TRIMEDX Holdings, LLC\Desktop\FF_DB\predict.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data["FORECAST_DATE"] = pd.date_range(
c:\Users\012356.CORPORATE\OneDrive - TRIMEDX Holdings, LLC\Desktop\FF_DB\predict.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data["FORECAST_DATE"] = pd.to_datetime(
c:\Users\012356.CORPORATE\OneDrive - TRIMEDX Holdings, LLC\Desktop\FF_DB\predict.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

In [10]:
np.mean(df_top_100['SEASON MAPE']), np.mean(df_top_100['GAME RSME'])

(0.2050942066754948, 8.086067218840094)